# Xml格式转换为Json

In [1]:
import xmltodict
import json
import collections
import re
from bs4 import BeautifulSoup as BS

import os

In [2]:
def parse_xml_string(file_path): 
    with open(file_path, 'r') as f:
        doc_string = f.read()
        
    soup = BS(doc_string)
    sentence_elements = soup.find_all('sentence')
    
    sentences = []
    for i, elem in enumerate(sentence_elements):
        elem = str(elem)
        elem = elem.replace('\n', '').replace('\t', '').replace('\r', '')
        
        pattern = re.compile('>[\u4e00-\u9fa50-9A-Za-z.，。！？：；“”"（）《》]+<')
        sentence = ' '.join([x.replace('<', '').replace('>', '') for x in pattern.findall(elem)])
        
        sentences.append(sentence)
    
    return sentences

In [3]:
def parse_xml(file_path, save_dir):
    json_sentences = []
    
    with open(file_path, 'r') as f:
        doc_string = f.read()
    doc = xmltodict.parse(doc_string)
    paragraphs = doc['Body']['Content']['Paragraph']

    if type(paragraphs) != list:
        assert type(paragraphs) == collections.OrderedDict
        paragraphs = [paragraphs]

    for i, paragraph in enumerate(paragraphs):
        try:
            sentences = paragraph['Sentence']
        except:
            continue

        if type(sentences) != list:
            sentences = [sentences]

        for j, sentence in enumerate(sentences):
            json_sentence = collections.OrderedDict()
            
            try:
                events = sentence['Event']
            except:
                assert type(sentence) == str
                json_sentences.append(json_sentence)
                continue

            if type(events) != list:
                assert type(events) == collections.OrderedDict
                events = [events]
            
            for e, event in enumerate(events):
                json_event = collections.OrderedDict()
                
                for k, v in event.items():
                    if k in ['@eid', '#text']:
                        continue
                        
                    if type(v) == collections.OrderedDict:
                        if k == 'Denoter':
                            json_event[k] = v
                        else:
                            try:
                                json_event[k] = v['#text']
                            except:
                                continue
                                
                    else:
                        json_event[k] = v
                
                json_sentence['event{}'.format(e)] = json_event
                
            json_sentences.append(json_sentence)
    
    
    raw_sentences = parse_xml_string(file_path)
    try:
        assert len(raw_sentences) == len(json_sentences)
    except:
        assert file_path == './CEC/食物中毒/印度发生假酒集体中毒事件.xml'
        del raw_sentences[3]
        assert len(raw_sentences) == len(json_sentences)
    
    for i, json_sentence in enumerate(json_sentences):
        json_sentence['sentence'] = raw_sentences[i]
    
    file_name = file_path.split('/')[-1].split('.xml')[0]
    with open('{}/{}.json'.format(save_dir, file_name), 'w') as f:
        json.dump(json_sentences, f, indent=4, ensure_ascii=False, sort_keys=True)

In [4]:
xml_files = []
for path, dir_list, file_list in os.walk('./CEC/'):
    for file_name in file_list:
        if '.xml' in file_name:
            xml_files.append(os.path.join(path, file_name))
len(xml_files)

332

In [5]:
for xml_file in xml_files:
    parse_xml(xml_file, save_dir='./CEC-xml2json/')

# 筛选可用句子，构造 Event Extraction 数据集

In [1]:
import json
import pandas as pd
import os
import jieba
import random

In [2]:
json_files = []
for path, dir_list, file_list in os.walk('./CEC-xml2json/'):
    for file_name in file_list:
        if '.json' in file_name:
            json_files.append(os.path.join(path, file_name))
len(json_files)

sentences = []
for json_file in json_files:
    with open(json_file, 'r') as f:
        sentences += json.load(f)
len(sentences)

2207

In [3]:
sentences[0]

{'event0': {'Denoter': {'#text': '火灾', '@did': 'd1', '@type': 'emergency'},
  'Location': '广州番禺市桥街兴泰路',
  'Object': '商铺',
  'Time': '2014年1月7日'},
 'event1': {'Denoter': {'#text': '烧死', '@did': 'd2', '@type': 'emergency'},
  'Participant': '从化女子'},
 'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！'}

In [4]:
def cut_sentence(text):
    cut_text = ''

    texts = text.split()
    for t in texts:
        cut_text += ' '.join(list(jieba.cut(t))) + ' '

    return cut_text[:-1]

In [5]:
print(sentences[2]['sentence'])
cut_sentence(sentences[2]['sentence'])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kz/g2361vfn397dqyxgt9n8bk6r0000gn/T/jieba.cache


广州番禺警方 媒体 通报 ： 1月7日晚21时40分 警方 群众 报警 ， 称 市桥街兴泰路 商铺 冒出 浓烟 。


Loading model cost 1.190 seconds.
Prefix dict has been built succesfully.


'广州 番禺 警方 媒体 通报 ： 1 月 7 日晚 21 时 40 分 警方 群众 报警 ， 称 市桥 街兴泰路 商铺 冒 出 浓烟 。'

In [6]:
valid_sentences = []

for sentence in sentences:
    # 没有event字段
    if len(sentence) == 1:
        continue
    
    valid_sentence = dict()
    
    text = sentence['sentence']
    cut_text = cut_sentence(text)
    words = cut_text.split()
    valid_sentence['sentence'] = text
    valid_sentence['sentence_words'] = cut_text
    
    triggers = []
    for key, value in sentence.items():
        if 'event' not in key:
            continue
            
        trigger = dict()
        
        # event trigger: Denoter 字段
        try:
            trigger['event'] = value['Denoter']['@type']
            # 去掉 thoughtevent 的事件
            if trigger['event'] == 'thoughtevent':
                continue
            trigger['event_trigger'] = value['Denoter']['#text']
        except:
            continue
        
        ## check trigger word ##
        if trigger['event_trigger'] not in words:
            continue

        # event arguments: Participant/Object 字段
        if 'Participant' in value.keys():
            participants = value['Participant']

            if type(participants) == list:
                ## check arguments word ##
                for participant in participants:
                    if participant not in words:
                        continue
                    
                    if 'event_arguments' not in trigger.keys():
                        trigger['event_arguments'] = [participant]
                    else:
                        trigger['event_arguments'].append(participant)
            else:
                assert type(participants) == str
                
                ## check arguments word ##
                if participants not in words:
                    continue
                    
                trigger['event_arguments'] = [participants]

        elif 'Object' in value.keys():
            participants = value['Object']
            
            ## check arguments word ##
            if participants not in words:
                continue
                    
            trigger['event_arguments'] = [participants]
            

        triggers.append(trigger)
    
    if len(triggers) == 0:
        continue
        
    valid_sentence['triggers'] = triggers
    valid_sentences.append(valid_sentence)

In [7]:
len(sentences), len(valid_sentences)

(2207, 1665)

In [8]:
valid_sentences[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': ['商铺'],
   'event_trigger': '火灾'}]}

In [9]:
with open('./dataset.json', 'w') as f:
    json.dump(valid_sentences, f, sort_keys=True, indent=4, ensure_ascii=False)

# Statistics

In [1]:
import json
import pandas as pd

In [2]:
with open('./dataset.json', 'r') as f:
    sentences = json.load(f)
len(sentences)

1665

In [3]:
sentences[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': '商铺',
   'event_trigger': '火灾',
   'index_event': 1,
   'index_event_arguments': 10,
   'index_event_trigger': 11}]}

In [4]:
sentences[1]

{'sentence': '广州番禺警方 媒体 通报 ： 1月7日晚21时40分 警方 群众 报警 ， 称 市桥街兴泰路 商铺 冒出 浓烟 。',
 'sentence_words': '广州 番禺 警方 媒体 通报 ： 1 月 7 日晚 21 时 40 分 警方 群众 报警 ， 称 市桥 街兴泰路 商铺 冒 出 浓烟 。',
 'triggers': [{'event': 'statement',
   'event_trigger': '称',
   'index_event': 6,
   'index_event_trigger': 18},
  {'event': 'statement',
   'event_trigger': '通报',
   'index_event': 6,
   'index_event_trigger': 4},
  {'event': 'operation',
   'event_trigger': '报警',
   'index_event': 3,
   'index_event_trigger': 16}]}

## 句子中 Event Trigger 数量统计

In [12]:
sentences[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': ['商铺'],
   'event_trigger': '火灾'}]}

In [13]:
triggers_num = [len(s['triggers']) for s in sentences]
len(triggers_num)

1665

In [14]:
triggers_df = pd.DataFrame({'triggers_num': triggers_num})
triggers_df.describe()

,triggers_num
count,1665.000000
mean,2.045646
std,1.298235
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,8.000000


### 1/1 比例（句子中只有一个事件）

In [15]:
len(triggers_df[triggers_df['triggers_num'] == 1])

765

In [16]:
len(triggers_df[triggers_df['triggers_num'] == 1]) / len(triggers_df)

0.4594594594594595

### 1/N 比例 （句子中有多个事件）

In [17]:
len(triggers_df[triggers_df['triggers_num'] > 1])

900

In [18]:
len(triggers_df[triggers_df['triggers_num'] > 1]) / len(triggers_df)

0.5405405405405406

In [19]:
sentences[triggers_num.index(8)]

{'sentence': '近日 ，八通网百姓热线，被撞男孩 父亲 发帖 描述 事发经过 ， 据 他 称 ， 11月14日早上6点40分左右 ， 13岁的儿子 骑 到 玉桥中学 上学 ， 行至 时 ， 被 一辆车牌号为京EY122的通州城管执法车（白色现代索纳塔） 撞伤 ， 儿子 被 拖行 了40余米 ， 幸亏 路人 和 玉桥中学的老师 相 救 ， 及时 送 到 潞河医院 抢救 。',
 'sentence_words': '近日 ， 八通网 百姓 热线 ， 被 撞 男孩 父亲 发帖 描述 事发 经过 ， 据 他 称 ， 11 月 14 日 早上 6 点 40 分 左右 ， 13 岁 的 儿子 骑 到 玉桥 中学 上学 ， 行至 时 ， 被 一辆 车牌号 为京 EY122 的 通州 城管 执法 车 （ 白色 现代 索纳塔 ） 撞伤 ， 儿子 被 拖行 了 40 余米 ， 幸亏 路 人 和 玉桥 中学 的 老师 相 救 ， 及时 送 到 潞河 医院 抢救 。',
 'triggers': [{'event': 'action', 'event_trigger': '上学'},
  {'event': 'statement', 'event_arguments': ['父亲'], 'event_trigger': '描述'},
  {'event': 'action', 'event_trigger': '救'},
  {'event': 'action', 'event_trigger': '送'},
  {'event': 'movement', 'event_trigger': '行至'},
  {'event': 'action', 'event_trigger': '抢救'},
  {'event': 'statement', 'event_arguments': ['他'], 'event_trigger': '称'},
  {'event': 'movement', 'event_arguments': ['儿子'], 'event_trigger': '拖行'}]}

## Event 类型统计

In [23]:
event_type = dict()

for sent in sentences:
    for trigger in sent['triggers']:
        t = trigger['event']
        if t not in event_type.keys():
            event_type[t] = 1
        else:
            event_type[t] += 1

len(event_type)

7

In [24]:
event_type

{'action': 639,
 'emergency': 599,
 'movement': 302,
 'operation': 764,
 'perception': 202,
 'stateChange': 415,
 'statement': 485}

## Event Argument 数量统计

In [26]:
sentences[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': ['商铺'],
   'event_trigger': '火灾'}]}

In [27]:
arguments_num = []

for sent in sentences:
    for trigger in sent['triggers']:
        if 'event_arguments' not in trigger.keys():
            arguments_num.append(0)
        else:
            arguments_num.append(len(trigger['event_arguments']))

len(arguments_num)

3406

In [28]:
arguments_df = pd.DataFrame({'num': arguments_num})
arguments_df.describe()

,num
count,3406.000000
mean,0.449501
std,0.497516
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [29]:
arguments_df['num'].value_counts()

0    1875
1    1531
Name: num, dtype: int64

In [31]:
arguments_df['num'].value_counts() / len(arguments_df)

0    0.550499
1    0.449501
Name: num, dtype: float64

In [32]:
# 把 event arguments 的类型，由list -> str
for sent in sentences:
    for trigger in sent['triggers']:
        if 'event_arguments' in trigger.keys():
            trigger['event_arguments'] = trigger['event_arguments'][0]

In [33]:
sentences[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': '商铺',
   'event_trigger': '火灾'}]}

In [34]:
with open('./dataset.json', 'w') as f:
    json.dump(sentences, f, sort_keys=True, indent=4, ensure_ascii=False)

## 最大句子长度：85

In [4]:
len(sentences)

1665

In [5]:
sentences[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': '商铺',
   'event_trigger': '火灾'}]}

In [6]:
nums = []
for piece in sentences:
    nums.append(len(piece['sentence_words'].split()))

In [7]:
len(nums)

1665

In [9]:
max(nums), nums.index(max(nums))

(85, 120)

In [10]:
sentences[120]

{'sentence': '近日 ，八通网百姓热线，被撞男孩 父亲 发帖 描述 事发经过 ， 据 他 称 ， 11月14日早上6点40分左右 ， 13岁的儿子 骑 到 玉桥中学 上学 ， 行至 时 ， 被 一辆车牌号为京EY122的通州城管执法车（白色现代索纳塔） 撞伤 ， 儿子 被 拖行 了40余米 ， 幸亏 路人 和 玉桥中学的老师 相 救 ， 及时 送 到 潞河医院 抢救 。',
 'sentence_words': '近日 ， 八通网 百姓 热线 ， 被 撞 男孩 父亲 发帖 描述 事发 经过 ， 据 他 称 ， 11 月 14 日 早上 6 点 40 分 左右 ， 13 岁 的 儿子 骑 到 玉桥 中学 上学 ， 行至 时 ， 被 一辆 车牌号 为京 EY122 的 通州 城管 执法 车 （ 白色 现代 索纳塔 ） 撞伤 ， 儿子 被 拖行 了 40 余米 ， 幸亏 路 人 和 玉桥 中学 的 老师 相 救 ， 及时 送 到 潞河 医院 抢救 。',
 'triggers': [{'event': 'action', 'event_trigger': '上学'},
  {'event': 'statement', 'event_arguments': '父亲', 'event_trigger': '描述'},
  {'event': 'action', 'event_trigger': '救'},
  {'event': 'action', 'event_trigger': '送'},
  {'event': 'movement', 'event_trigger': '行至'},
  {'event': 'action', 'event_trigger': '抢救'},
  {'event': 'statement', 'event_arguments': '他', 'event_trigger': '称'},
  {'event': 'movement', 'event_arguments': '儿子', 'event_trigger': '拖行'}]}

# Train/Test 划分

## 按照event trigger的数量来分层抽样

In [4]:
import json
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
with open('./dataset.json', 'r') as f:
    dataset = json.load(f)
    
len(dataset)

1665

In [3]:
dataset[0]

{'sentence': '2014年1月7日 广州番禺市桥街兴泰路 商铺 火灾 ， 从化女子 烧死 ！',
 'sentence_words': '2014 年 1 月 7 日 广州 番禺市 桥街 兴泰路 商铺 火灾 ， 从化 女子 烧死 ！',
 'triggers': [{'event': 'emergency',
   'event_arguments': '商铺',
   'event_trigger': '火灾',
   'index_event': 1,
   'index_event_arguments': 10,
   'index_event_trigger': 11}]}

In [5]:
triggers_num = [len(p['triggers']) for p in dataset]
len(triggers_num)

1665

In [6]:
triggers_num[:5]

[1, 3, 3, 1, 4]

In [7]:
set(triggers_num)

{1, 2, 3, 4, 5, 6, 7, 8}

In [8]:
y = np.arange(len(dataset))
y.shape

(1665,)

In [16]:
train_index, test_index = train_test_split(y, test_size=0.2, stratify=triggers_num, random_state=0)
train_index.shape, test_index.shape

((1332,), (333,))

In [18]:
def check_triggers_num(indexes):
    num = np.array(triggers_num)
    
    chosen_num = num[indexes]
    events = list(set(chosen_num))
    events.sort()
    
    num2len = dict()
    for e in events:
        num2len[e] = len(chosen_num[chosen_num==e])
    
    print(num2len)

In [19]:
check_triggers_num(y)

{1: 765, 2: 429, 3: 249, 4: 131, 5: 55, 6: 20, 7: 11, 8: 5}


In [20]:
check_triggers_num(train_index)

{1: 612, 2: 343, 3: 199, 4: 105, 5: 44, 6: 16, 7: 9, 8: 4}


In [21]:
check_triggers_num(test_index)

{1: 153, 2: 86, 3: 50, 4: 26, 5: 11, 6: 4, 7: 2, 8: 1}
